In [6]:
# 安装包(Python >= 3.7)：pip install qianfan
import os
import pandas as pd
import qianfan
from sklearn.metrics import accuracy_score

# 设置环境变量
os.environ["QIANFAN_ACCESS_KEY"] = "ALTAK6cCFf3NbY2fmJknPx0adl"
os.environ["QIANFAN_SECRET_KEY"] = "e49bfe6b4006419d98f0fd3bdb995b63"

# 加载Excel文件
file_path = '/Users/clairewang/Desktop/trust_or_not/标注.xlsx'
df = pd.read_excel(file_path)

# 初始化模型
chat_comp = qianfan.ChatCompletion()

# 定义计算相似度的函数
def calculate_accuracy(predictions, targets):
    return accuracy_score(targets, predictions)

# 定义生成响应的函数
def generate_response(text, temperature):
    prompt_text = f"忘掉所有之前的指令。你现在是一位金融、管理与会计专家。我会给出一个投资者对上市公司提问的文本，你需要回答这个提问文本是暗示投资者信任还是不信任这家公司。请在“信任”、“不信任”、“未知”中只选择一个选项，并且不要提供任何额外的回答：{text}"
    try:
        response = chat_comp.do(
            endpoint="ernie-speed-128k",
            user_id="user",
            messages=[
                {
                    "role": "user",
                    "content": prompt_text
                }
            ],
            temperature=temperature,
            top_p=0,
            penalty_score=1
        )
        response_text = response['body']['result']  # 提取实际文本
        print(f"Temperature: {temperature}, Text: {text}, Response: {response_text}")
        return response_text
    except Exception as e:
        error_message = f"错误: {e}"
        print(error_message)
        return error_message

# 提取关键词的函数
def extract_keyword(response):
    if response.startswith("信任"):
        return "信任"
    elif response.startswith("不信任"):
        return "不信任"
    elif response.startswith("未知"):
        return "未知"
    else:
        return "未知"  # 如果没有匹配到关键词，默认返回"未知"

# 尝试特定的temperature值，并找到最佳值
best_temperature = None
best_accuracy = 0
temperature_values = [0.2, 0.4, 0.6, 0.8, 1.0]  # 特定的温度值

for temp in temperature_values:
    predictions = []
    for text in df['提问']:
        response = generate_response(text, temp)
        keyword = extract_keyword(response)
        predictions.append(keyword)
        print(f"提取的关键词: {keyword}")
    
    # 计算当前temperature值的准确率
    accuracy = calculate_accuracy(predictions, df['信任程度'])
    print(f"Temperature: {temp}, Accuracy: {accuracy}")
    
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_temperature = temp

print(f"最佳的temperature值为: {best_temperature}, 准确率为: {best_accuracy}")


Temperature: 0.2, Text: 请问，截至2023年12月20日收盘，贵司的股东人数是多少？谢谢！, Response: 信任。
提取的关键词: 信任
Temperature: 0.2, Text: 董秘您好，请问今年是否有分红派息计划？, Response: 信任。

这个问题是询问公司是否有分红派息计划，表明投资者对公司的财务状况和盈利能力感兴趣，并期望了解公司是否愿意将部分利润返还给股东。因此，这个问题暗示投资者信任这家公司，并期待从中获得回报。
提取的关键词: 信任
Temperature: 0.2, Text: 您好，今年贵公司内部控制（控制环境 、 风险评估 、 控制活动 、信息与沟通、监督）有相关信息披露吗？在哪里查看呢？我这边打算写有关贵公司内部控制对会计信息质量影响这样的文章，需要贵公司内部控制的最新情况，谢谢您的解答。, Response: 信任。

从提问中可以看出，提问者对于公司的内部控制情况持有关注和兴趣，并询问了相关的信息披露和查看方式。这表明提问者对公司有一定的信任基础，希望通过了解公司的内部控制情况来进一步了解公司的运营和管理状况。因此，这个提问文本暗示投资者信任这家公司。
提取的关键词: 信任
Temperature: 0.2, Text: 您好，我看到贵公司华证的ESG评分仅为CCC，处于行业末端且与同互联网行业的软通动力相差甚远。请问应如何看待ESG对信息安全业务的相关性或影响？有无提升ESG竞争力的计划？谢谢！, Response: 信任。
提取的关键词: 信任
Temperature: 0.2, Text: 董秘：你好！国华鸿智成立已经大半年，请问公司开发出的智能360环视记录仪和车辆预警系统是否上市，公司开发这两种产品是针对C端汽车个人加装市场，还是谋求进入品牌汽车厂商供应链市场？目前进展如何？, Response: 信任。
提取的关键词: 信任
Temperature: 0.2, Text: 贵公司在车联网安全上有产品吗？贵公司和宝马汽车有合作意向吗？, Response: 信任。从提问文本来看，投资者对贵公司的车联网安全产品和与宝马汽车的合作意向表现出一定的关注和兴趣，这通常被视为对贵公司的信任表现。提问者想要了解公司的具体情况和合作意向，这暗示了对公司能力的信任和对未来合作的期待。
提取的关键词: 信